In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_validate

#Model from Scikit-Learn Regression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import VotingRegressor

In [ ]:
models = {
"LinearRegression": LinearRegression(),
"LGBMRegressor":LGBMRegressor(),
"GradientBoostingRegressor":GradientBoostingRegressor(random_state=0),
"ExtraTreesRegressor": ExtraTreesRegressor(n_jobs=-1),
"RandomForestRegressor": RandomForestRegressor(n_jobs=-1),
}

dic =  {"Model":[],"RMSE_Train":[]}

def fit_and_evaluate(models, X_train, X_val, y_train, y_val):
    for name, model in models.items(): 
        model.fit(X_train, y_train)
        y_train_pre = model.predict(X_val)
        rmse  = np.sqrt(mean_squared_error(y_val, y_train_pre))
        print("--------------------------------------------------------------")
        print("Model:", name)
        print("-----Training Data Evalution-----")
        print("RMSE: ", rmse)
        dic["Model"].append(name)
        dic["RMSE_Train"].append(rmse)

In [ ]:
train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
train_df

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.drop(['Id'],axis=1,inplace=True)
test_df.drop(['Id'],axis=1,inplace=True)

In [ ]:
y = train_df['Pawpularity']
train_df.drop('Pawpularity',axis=1,inplace=True)

In [ ]:
# Split train - validate data
X_train,X_val,y_train,y_val = train_test_split(train_df,y,test_size=0.2)

In [ ]:
fit_and_evaluate(models, X_train, X_val, y_train, y_val)

In [ ]:
final_data = pd.DataFrame(dic)
final_data.sort_values("RMSE_Train", axis = 0, ascending = True)

In [ ]:
best_model = LinearRegression()
best_model.fit(X_train, y_train)

In [ ]:
def model_result(clf,x_val,y_val):
    y_pred = clf.predict(x_val)
    print("RMSE: ")
    print(np.sqrt(mean_squared_error(y_val,y_pred)))

In [ ]:
clf1 = LinearRegression()
clf2 = LGBMRegressor()
clf3 = GradientBoostingRegressor()
clf4 = RandomForestRegressor()
clf5 = ExtraTreesRegressor()

vot_clf = VotingRegressor(estimators=[('lr_clf', clf1), ('lgbm_clf', clf2), ('gb_clf', clf3)])

In [ ]:
vot_clf = vot_clf.fit(X_train, y_train)

In [ ]:
model_result(vot_clf, X_val, y_val)

In [ ]:
model_result(best_model, X_val, y_val)

In [ ]:
predicts = best_model.predict(test_df)

In [ ]:
df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

submission_df = pd.DataFrame()
submission_df['Id'] = df['Id']
submission_df['Pawpularity'] = predicts

submission_df.to_csv('submission.csv', index=False)